El objetivo de este notebook es hacer una implementación rápida de un Decision Tree y ganar intuición sobre la forma de tomar las decisiones.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [2]:
import numpy as np
import joblib
import sys
import pandas as pd
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from sklearn.model_selection import PredefinedSplit, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import time
import pickle

In [3]:
# Importar los datos
features = joblib.load("/content/drive/My Drive/tesis_Molina/estructura_codigos/data/datos_definitivos/FEATURES_W200_I50.joblib")
label = joblib.load("/content/drive/My Drive/tesis_Molina/estructura_codigos/data/datos_definitivos/label_W200_I50.joblib")

features_array = np.array(features)
label_array = np.array(label)

# Grid Search

## Separación de datos por sujeto

In [ ]:
%cd "/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/particiones_1subTest"


full_pipeline = Pipeline([
    ('model', DecisionTreeClassifier())
])

param_grid = {
    'model__max_depth': [5, 10, 15, 20, None],  # , 10, 15, 20, None
}

# defino las metricas que quiero guardar del gridsearch
scorers = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro')
}


pipelines_CV = []   # guardo los objetos gridsearch ajustados de cada experimento. voy a tener uno por fold (siempre mando un fold a test)
particiones_CV = [] # lista de df. guardo la particion en folds usada en cada caso (los numeros pueden no corresponderse con los originales, dependiendo qué fold manda a test)

for i in range(0, 10): # aca decido qué sujeto va para test
  # Importo particiones
  folds_ind = joblib.load('folds_W200_I50_sujetoTest_' + str(i+1) + '.pkl') # lista de listas. en cada elemento tengo la lista de indices de un fold
  sujetos_array = joblib.load('arraySujetos_folds_W200_I50_sujetoTest_' + str(i+1) + '.pkl')   # lista con todos los sujetos, donde el primero corresponde al de test

  # por el criterio elegido, se que el primer fold es para test, asi que lo saco
  fold_test = folds_ind[0]

  # ahora lo quito para no armar entrevero
  folds_ind = folds_ind[1:]
  sujetos_CV = sujetos_array[1:]   # me quedo solo con los sujetos que voy usar para hacer CV
  # ahora tengo los 9 folds que voy a usar en la CV

  # construyo una lista que indica a que fold pertenece cada dato
  test_fold = []
  # concateno y aplano la lista de arreglos
  folds_ind_aplanado = np.concatenate(folds_ind).ravel()
  for k in range(len(folds_ind)):
    for j in range(len(folds_ind[k])):
      test_fold.append(k)
  # folds_ind_aplanado guarda el indice de dato de cada elemento de test_fold
  # lo guardo en un DataFrame para que quede mas claro
  data = {
      'indice_dato': folds_ind_aplanado,
      'fold_aux': test_fold
  }
  # guardo data
  particiones_CV.append(pd.DataFrame(data))

  # procedo a hacer la validación cruzada con gridsearch

  # creo el objeto PredefinedSplit
  ps = PredefinedSplit(test_fold)

  # preparo los datos
  X_cv = features_array[folds_ind_aplanado, :]
  y_cv = label_array[folds_ind_aplanado, 1]

  # implemento GridSearchCV con el pipeline y PredefinedSplit
  grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid, cv=ps, scoring=scorers, refit=False, return_train_score=True)

  # ajusto GridSearchCV
  grid_search.fit(X_cv, y_cv)

  pipelines_CV.append(grid_search)
  joblib.dump(grid_search, 'gs_dt_PredefinedSplit' + str(i) + '_v2.pkl')
  # pipelines_CV.append(joblib.load('ti_gs_dt_PredefinedSplit' + str(i) + '.pkl'))
joblib.dump(pipelines_CV, 'gs_dt_PredefinedSplit_v2".pkl')
joblib.dump(particiones_CV, 'dt_PredefinedSplit_v2.pkl')

/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/particiones_1subTest


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

In [ ]:
# Guardar la lista de objetos en un archivo
with open('ti_act__gs_dt_PredefinedSplit.pkl', 'wb') as file:
    pickle.dump(pipelines_CV, file)

In [ ]:
# Obtener los resultados de GridSearchCV
results = grid_search.cv_results_

# Crear un DataFrame con los resultados
df_results = pd.DataFrame(results)

df_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__max_depth,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,...,split1_train_min_inference_time,split2_train_min_inference_time,split3_train_min_inference_time,split4_train_min_inference_time,split5_train_min_inference_time,split6_train_min_inference_time,split7_train_min_inference_time,split8_train_min_inference_time,mean_train_min_inference_time,std_train_min_inference_time
0,10.807443,1.382616,2.764365,0.531135,5,{'model__max_depth': 5},0.242724,0.233207,0.270791,0.288112,...,0.00007,0.00007,0.00007,0.000071,0.00007,0.00007,0.000070,0.00007,0.00007,3.066550e-07
1,18.872635,0.469696,2.640205,0.453585,10,{'model__max_depth': 10},0.283864,0.283630,0.289640,0.338466,...,0.00007,0.00007,0.00007,0.000070,0.00007,0.00007,0.000070,0.00007,0.00007,1.123916e-07
2,25.545935,1.064951,2.443999,0.421587,15,{'model__max_depth': 15},0.277007,0.272645,0.271367,0.277949,...,0.00007,0.00007,0.00007,0.000069,0.00007,0.00007,0.000069,0.00007,0.00007,2.456670e-07
3,28.946718,0.618056,2.578693,0.468227,20,{'model__max_depth': 20},0.274722,0.247614,0.282446,0.284570,...,0.00007,0.00007,0.00007,0.000070,0.00007,0.00007,0.000070,0.00007,0.00007,1.184711e-07
4,30.785708,0.829980,2.425455,0.462405,None,{'model__max_depth': None},0.253238,0.246353,0.278705,0.264244,...,0.00007,0.00007,0.00007,0.000070,0.00007,0.00007,0.000070,0.00007,0.00007,1.350779e-07


## Separación de datos por repetición

In [ ]:
%cd "/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/particiones_1subTest"

folds_ind = joblib.load("10foldsXRepeticion_W200_I50.pkl")

folds_ind_array = []

for i in range(len(folds_ind)):
  folds_ind_array.append(np.array(folds_ind[i]))

folds_ind = folds_ind_array.copy()

full_pipeline = Pipeline([
    ('model', DecisionTreeClassifier())
])

param_grid = {
    'model__max_depth': [5, 10, 15, 20, None],
}

# defino las metricas que quiero guardar del gridsearch
scorers = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro')
}

pipelines_CV = []   # guardo los objetos gridsearch ajustados de cada experimento. voy a tener uno por fold (siempre mando un fold a test)
particiones_CV = [] # lista de df. guardo la particion en folds usada en cada caso (los numeros pueden no corresponderse con los originales, dependiendo qué fold manda a test)

for i in range(0, 10): # aca decido qué sujeto va para test
  # # Importo particiones
  # folds_ind = joblib.load('folds_W200_I50_sujetoTest_' + str(i+1) + '.pkl') # lista de listas. en cada elemento tengo la lista de indices de un fold
  # sujetos_array = joblib.load('arraySujetos_folds_W200_I50_sujetoTest_' + str(i+1) + '.pkl')   # lista con todos los sujetos, donde el primero corresponde al de test
  folds_ind = folds_ind_array.copy()
  # por el criterio elegido, se que el primer fold es para test, asi que lo saco
  fold_test = folds_ind[i]

  # ahora lo quito para no armar entrevero
  folds_ind.pop(i)
  # print(folds_ind)
#   # sujetos_CV = sujetos_array[1:]   # me quedo solo con los sujetos que voy usar para hacer CV
#   # ahora tengo los 9 folds que voy a usar en la CV

  # construyo una lista que indica a que fold pertenece cada dato
  test_fold = []
  # concateno y aplano la lista de arreglos
  folds_ind_aplanado = np.concatenate(folds_ind).ravel()
  for k in range(len(folds_ind)):
    for j in range(len(folds_ind[k])):
      test_fold.append(k)
  # folds_ind_aplanado guarda el indice de dato de cada elemento de test_fold
  # lo guardo en un DataFrame para que quede mas claro
  data = {
      'indice_dato': folds_ind_aplanado,
      'fold_aux': test_fold
  }
  # guardo data
  particiones_CV.append(pd.DataFrame(data))

  # procedo a hacer la validación cruzada con gridsearch

  # creo el objeto PredefinedSplit
  ps = PredefinedSplit(test_fold)

  # preparo los datos
  X_cv = features_array[folds_ind_aplanado, :]
  y_cv = label_array[folds_ind_aplanado, 1]

  # implemento GridSearchCV con el pipeline y PredefinedSplit
  grid_search = GridSearchCV(estimator=full_pipeline, param_grid=param_grid, cv=ps, scoring=scorers, refit=False, return_train_score=True)

  # ajusto GridSearchCV
  grid_search.fit(X_cv, y_cv)

  pipelines_CV.append(grid_search)
  # joblib.dump(grid_search, 'gs_dt_PredefinedSplit' + str(i) + '.pkl')

joblib.dump(pipelines_CV, 'gs_dt_PredefinedSplit_sep_rep.pkl')
joblib.dump(particiones_CV, 'ps_dt_PredefinedSplit_sep_rep.pkl')

In [ ]:
# Obtener los resultados de GridSearchCV
results = grid_search.cv_results_

# Crear un DataFrame con los resultados
df_results = pd.DataFrame(results)

df_results

# Experimentos 9 train/1 test



## Separación de datos por sujeto

In [ ]:
%cd "/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/hello_world/resultados_preliminares/con_PS"
hyperparams = joblib.load("hyperparams_sub_dt.pkl")

print(hyperparams)

/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/hello_world/resultados_preliminares/con_PS
[10, 10, 10, 10, 10, 15, 10, 5, 10, 10]


In [ ]:
%cd "/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/hello_world/resultados_preliminares/con_PS"
hyperparams = joblib.load("hyperparams_sub_dt.pkl")

%cd "/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/particiones_1subTest"

precision_train = []
accuracy_train = []
recall_train = []
precision_test = []
accuracy_test = []
recall_test = []
guardar_modelos = []
sujetos_test = []
tiempos_predict = []  # mean, std, max, min

for i in range(0, 10):
    folds_ind = joblib.load('folds_W200_I50_sujetoTest_' + str(i + 1) + '.pkl')
    sujetos_array = joblib.load('arraySujetos_folds_W200_I50_sujetoTest_' + str(i + 1) + '.pkl')
    sujetos_test.append(sujetos_array[0])
    fold_test = folds_ind[0]
    folds_ind = folds_ind[1:]
    folds_ind_train_array = np.concatenate(folds_ind).ravel()
    X_train = features_array[folds_ind_train_array, :]
    y_train = label_array[folds_ind_train_array, 1]
    X_test = features_array[fold_test, :]
    y_test = label_array[fold_test, 1]

    # Crear una nueva instancia del pipeline en cada iteración
    full_pipeline = Pipeline([
        ('dt_classifier', DecisionTreeClassifier(max_depth=hyperparams[i]))
    ])
    print(f"Fold {i+1}: Setting max_depth to {hyperparams[i]}")

    full_pipeline.fit(X_train, y_train)
    y_pred_train = full_pipeline.predict(X_train)
    start_time = time.time()
    y_pred_test = full_pipeline.predict(X_test)
    end_time = time.time()
    training_time = end_time - start_time
    tiempos_predict.append(training_time)

    precision_train.append(precision_score(y_train, y_pred_train, average='macro'))
    precision_test.append(precision_score(y_test, y_pred_test, average='macro'))
    accuracy_train.append(accuracy_score(y_train, y_pred_train))
    accuracy_test.append(accuracy_score(y_test, y_pred_test))
    recall_train.append(recall_score(y_train, y_pred_train, average='macro'))
    recall_test.append(recall_score(y_test, y_pred_test, average='macro'))
    guardar_modelos.append(full_pipeline)


/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/hello_world/resultados_preliminares/con_PS
/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/particiones_1subTest
Fold 1: Setting max_depth to 10
Fold 2: Setting max_depth to 10
Fold 3: Setting max_depth to 10
Fold 4: Setting max_depth to 10
Fold 5: Setting max_depth to 10
Fold 6: Setting max_depth to 15
Fold 7: Setting max_depth to 10
Fold 8: Setting max_depth to 5


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 9: Setting max_depth to 10
Fold 10: Setting max_depth to 10


In [ ]:
%cd "/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/hello_world/resultados_preliminares/con_PS"

# Guardo modelos
joblib.dump(guardar_modelos, 'dt_HW_NP_10_sep_sub_max_depth_trainTest_modelos.pkl')
# # Guardo hiperparametros usados
# joblib.dump(hyperparams, 'dt_HW_NP_10_sep_sub_max_depth_trainTest_hyperparams.pkl')
# Guardo las metricas
data = {'sujeto_test': sujetos_test,
        'accuracy_train': accuracy_train,
        'precision_train': precision_train,
        'recall_train': recall_train,
        'accuracy_test': accuracy_test,
        'precision_test': precision_test,
        'recall_test': recall_test,
        'mean_score_time_x_dato': np.array(tiempos_predict)/6453}
data = pd.DataFrame(data)
joblib.dump(data, 'dt_HW_NP_10_sep_sub_max_depth_trainTest_metricas.pkl')

/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/hello_world/resultados_preliminares/con_PS


['dt_HW_NP_10_sep_sub_max_depth_trainTest_metricas.pkl']

In [ ]:
data

,sujeto_test,accuracy_train,precision_train,recall_train,accuracy_test,precision_test,recall_test,mean_score_time_x_dato
0,0,0.485263,0.565098,0.481501,0.211227,0.228426,0.209660,7.114133e-07
1,1,0.472457,0.527696,0.467059,0.252476,0.309684,0.253101,1.608374e-06
2,2,0.484272,0.557009,0.479847,0.267063,0.334305,0.263951,8.774528e-07
3,3,0.479940,0.542817,0.475958,0.272662,0.327875,0.271716,1.011977e-06
4,4,0.472877,0.546083,0.467922,0.282107,0.364119,0.279357,7.033958e-07
5,5,0.698340,0.724519,0.696815,0.194327,0.221874,0.192486,9.045349e-07
6,6,0.477691,0.534323,0.474523,0.228357,0.232020,0.230757,1.037618e-06
7,7,0.300678,0.390074,0.292139,0.254704,0.333291,0.249713,7.509095e-07
8,8,0.497198,0.543623,0.491006,0.258317,0.282386,0.256093,6.262875e-07
9,9,0.475823,0.529529,0.471195,0.157312,0.192741,0.162895,1.379451e-06


## Separación de datos por repetición

In [ ]:
%cd "/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/hello_world/resultados_preliminares/con_PS"
hyperparams = joblib.load("hyperparams_rep_dt.pkl")

print(hyperparams)

/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/hello_world/resultados_preliminares/con_PS
[15, 15, 15, 15, 15, 15, 15, 15, 15, 15]


In [ ]:
%cd "/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/hello_world/resultados_preliminares/con_PS"
hyperparams = joblib.load("hyperparams_rep_dt.pkl")

%cd "/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/particiones_1subTest"

folds_ind = joblib.load("10foldsXRepeticion_W200_I50.pkl")

folds_ind_array = []

for i in range(len(folds_ind)):
  folds_ind_array.append(np.array(folds_ind[i]))

folds_ind = folds_ind_array.copy()

precision_train = []
accuracy_train = []
recall_train = []
precision_test = []
accuracy_test = []
recall_test = []
guardar_modelos = []
sujetos_test = []
tiempos_predict = []

folds_num = np.arange(0,10)

for i in range(0, 10): # aca decido qué sujeto va para test
  # # Importo particiones
  # folds_ind = joblib.load('folds_W200_I50_sujetoTest_' + str(i+1) + '.pkl') # lista de listas. en cada elemento tengo la lista de indices de un fold
  # sujetos_array = joblib.load('arraySujetos_folds_W200_I50_sujetoTest_' + str(i+1) + '.pkl')   # lista con todos los sujetos, donde el primero corresponde al de test
  sujetos_test.append(folds_num[i])
  folds_ind = folds_ind_array.copy()
  # por el criterio elegido, se que el primer fold es para test, asi que lo saco
  fold_test = folds_ind[i]

  # ahora lo quito para no armar entrevero
  folds_ind.pop(i)
  # print(folds_ind)
#   # sujetos_CV = sujetos_array[1:]   # me quedo solo con los sujetos que voy usar para hacer CV
#   # ahora tengo los 9 folds que voy a usar en la CV

  # construyo una lista que indica a que fold pertenece cada dato

  folds_ind_train_array = np.concatenate(folds_ind).ravel()

  X_train = features_array[folds_ind_train_array, :]
  y_train = label_array[folds_ind_train_array, 1]
  X_test = features_array[fold_test, :]
  y_test = label_array[fold_test, 1]

  # Crear una nueva instancia del pipeline en cada iteración
  full_pipeline = Pipeline([
      ('dt_classifier', DecisionTreeClassifier(max_depth=hyperparams[i]))
  ])
  print(f"Fold {i+1}: Setting max_depth to {hyperparams[i]}")

  full_pipeline.fit(X_train, y_train)
  y_pred_train = full_pipeline.predict(X_train)
  start_time = time.time()
  y_pred_test = full_pipeline.predict(X_test)
  end_time = time.time()
  training_time = end_time - start_time
  tiempos_predict.append(training_time)

  precision_train.append(precision_score(y_train, y_pred_train, average='macro'))
  precision_test.append(precision_score(y_test, y_pred_test, average='macro'))
  accuracy_train.append(accuracy_score(y_train, y_pred_train))
  accuracy_test.append(accuracy_score(y_test, y_pred_test))
  recall_train.append(recall_score(y_train, y_pred_train, average='macro'))
  recall_test.append(recall_score(y_test, y_pred_test, average='macro'))
  guardar_modelos.append(full_pipeline)

/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/hello_world/resultados_preliminares/con_PS
/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/particiones_1subTest
Fold 1: Setting max_depth to 15
Fold 2: Setting max_depth to 15
Fold 3: Setting max_depth to 15
Fold 4: Setting max_depth to 15
Fold 5: Setting max_depth to 15
Fold 6: Setting max_depth to 15
Fold 7: Setting max_depth to 15
Fold 8: Setting max_depth to 15
Fold 9: Setting max_depth to 15
Fold 10: Setting max_depth to 15


In [ ]:
%cd "/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/hello_world/resultados_preliminares/con_PS"

# Guardo modelos
joblib.dump(guardar_modelos, 'dt_HW_NP_10_sep_rep_max_depth_trainTest_modelos.pkl')
# # Guardo hiperparametros usados
# joblib.dump(hyperparams, 'dt_HW_NP_10_sep_sub_max_depth_trainTest_hyperparams.pkl')
# Guardo las metricas
data = {'sujeto_test': sujetos_test,
        'accuracy_train': accuracy_train,
        'precision_train': precision_train,
        'recall_train': recall_train,
        'accuracy_test': accuracy_test,
        'precision_test': precision_test,
        'recall_test': recall_test,
        'mean_score_time_x_dato': np.array(tiempos_predict)/6453}
data = pd.DataFrame(data)
joblib.dump(data, 'dt_HW_NP_10_sep_rep_max_depth_trainTest_metricas.pkl')

/content/drive/My Drive/repo_tesis/src/ronda_experimentos_1/hello_world/resultados_preliminares/con_PS


['dt_HW_NP_10_sep_rep_max_depth_trainTest_metricas.pkl']

In [ ]:
data

,sujeto_test,accuracy_train,precision_train,recall_train,accuracy_test,precision_test,recall_test,mean_score_time_x_dato
0,0,0.692136,0.716463,0.689511,0.415525,0.438808,0.415356,8.499643e-07
1,1,0.692259,0.720394,0.689803,0.427403,0.451274,0.427051,8.377718e-07
2,2,0.691527,0.717339,0.689006,0.377585,0.396555,0.376389,1.109147e-06
3,3,0.695715,0.713546,0.692866,0.400032,0.416553,0.401162,8.549152e-07
4,4,0.688201,0.712903,0.685616,0.431412,0.449330,0.426587,8.506663e-07
5,5,0.682130,0.708329,0.679411,0.422504,0.443510,0.422168,9.044241e-07
6,6,0.685313,0.705145,0.682992,0.404127,0.414489,0.401917,1.091523e-06
7,7,0.698180,0.718720,0.695708,0.432792,0.443250,0.433314,1.090267e-06
8,8,0.710561,0.722627,0.708116,0.412711,0.422441,0.410811,5.932570e-07
9,9,0.699995,0.713748,0.697993,0.424316,0.438010,0.427135,6.305734e-07


# Backup

In [ ]:
# ESTO ES JUNTANDO TODOS LOS RESULTADOS EN UN DATAFRAME

# Obtener los resultados de GridSearchCV
results = grid_search.cv_results_

# Crear una lista para almacenar los resultados
fold_results = []

# Obtener el número de splits (folds)
n_splits = len([key for key in results.keys() if key.startswith('split') and key.endswith('_test_score')])

# Iterar sobre los resultados para cada combinación de hiperparámetros
for i in range(len(results['params'])):
    for fold in range(n_splits):
        fold_result = {
            'Hyperparameter': results['param_dt_classifier__max_depth'][i],  # Valor del hiperparámetro
            'Fold': fold,  # Número de fold
            'Train Accuracy': results[f'split{fold}_train_score'][i],  # Puntaje de entrenamiento
            'Validation Accuracy': results[f'split{fold}_test_score'][i]  # Puntaje de validación
        }
        fold_results.append(fold_result)

# Crear un DataFrame con los resultados
df_results = pd.DataFrame(fold_results)

df_results

In [ ]:
# Crear un diccionario para almacenar los DataFrames
hyperparam_results = {}

# Obtener el número de splits (folds)
n_splits = len([key for key in results.keys() if key.startswith('split') and key.endswith('_test_score')])

# Iterar sobre los resultados para cada combinación de hiperparámetros
for i in range(len(results['params'])):
    param_value = results['param_dt_classifier__max_depth'][i]
    fold_results = []
    for fold in range(n_splits):
        fold_result = {
            'Fold': fold,  # Número de fold
            'train_accuracy': results[f'split{fold}_train_score'][i],  # Puntaje de entrenamiento
            'val_accuracy': results[f'split{fold}_test_score'][i]  # Puntaje de validación
        }
        fold_results.append(fold_result)

    # Crear un DataFrame para esta combinación de hiperparámetros
    df_results = pd.DataFrame(fold_results)

    # Añadir el DataFrame al diccionario
    hyperparam_results[f"max_depth={param_value}"] = df_results

# Mostrar los resultados para cada hiperparámetro
for param, df in hyperparam_results.items():
    print(f"Results for {param}:")
    print(df)
    print("\n")

In [ ]:
hyperparam_results['max_depth=10']

In [ ]:
# Armo una grafica similar a la que tenia antes
plt.figure()
plt.plot(hyperparam_results['max_depth=5'].train_accuracy, '.-')
plt.plot(hyperparam_results['max_depth=10'].train_accuracy, '.-')
plt.plot(hyperparam_results['max_depth=15'].train_accuracy, '.-')
plt.plot(hyperparam_results['max_depth=20'].train_accuracy, '.-')

plt.plot(hyperparam_results['max_depth=5'].val_accuracy, '-x')
plt.plot(hyperparam_results['max_depth=10'].val_accuracy, '-x')
plt.plot(hyperparam_results['max_depth=15'].val_accuracy, '-x')
plt.plot(hyperparam_results['max_depth=20'].val_accuracy, '-x')

plt.legend(['train 5', 'train 10', 'train 15', 'train 20','val 5', 'val 10', 'val 15', 'val 20'])

In [ ]:
# Crear un diccionario para almacenar los DataFrames
hyperparam_results = {}

# Obtener el número de splits (folds)
n_splits = len([key for key in results.keys() if key.startswith('split') and key.endswith('_test_score')])

# Iterar sobre los resultados para cada combinación de hiperparámetros
for i in range(len(results['params'])):
    param_value = results['params'][i]  # Obtener la combinación de hiperparámetros
    fold_results = []
    for fold in range(n_splits):
        fold_result = {
            'Fold': fold,  # Número de fold
            'train_accuracy': results[f'split{fold}_train_score'][i],  # Puntaje de entrenamiento
            'val_accuracy': results[f'split{fold}_test_score'][i],  # Puntaje de validación
            'Hyperparameters': str(param_value)  # Combinación de hiperparámetros como cadena de texto
        }
        fold_results.append(fold_result)

    # Crear un DataFrame para esta combinación de hiperparámetros
    df_results = pd.DataFrame(fold_results)

    # Añadir el DataFrame al diccionario
    hyperparam_results[f"params_{i}"] = df_results

In [ ]:
# Armo una grafica similar a la que tenia antes
plt.figure()
plt.plot(hyperparam_results['params_0'].train_accuracy, '.-')
plt.plot(hyperparam_results['params_1'].train_accuracy, '.-')
plt.plot(hyperparam_results['params_2'].train_accuracy, '.-')
plt.plot(hyperparam_results['params_3'].train_accuracy, '.-')

plt.plot(hyperparam_results['params_0'].val_accuracy, '-x')
plt.plot(hyperparam_results['params_1'].val_accuracy, '-x')
plt.plot(hyperparam_results['params_2'].val_accuracy, '-x')
plt.plot(hyperparam_results['params_3'].val_accuracy, '-x')

plt.legend(['train 5', 'train 10', 'train 15', 'train 20','val 5', 'val 10', 'val 15', 'val 20'])